In [6]:
# Dependencies
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import requests
import pprint

### NASA Mars News: 
#### Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 

In [17]:
# This step opens a new chrome window that will be driven by code in the next few cells
# It also adds another Chrome icon on the taskbar because it's using chromedriver instead of chrome
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [18]:
# This step navigates the chromebrowser window to NASA News site & runs BeautifulSoup to parse the site's HTML
nasa_url = "https://mars.nasa.gov/news/"
browser.visit(nasa_url)
sleep(3) # This is a manual delay that prevents a "Race Condition" with JavaScript and this script competing for resources
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [24]:
# Grab the latest headline from NASA Mars News
nasa_headline = soup.find_all("div", class_="content_title")[1].get_text()
nasa_headline

'NASA Moves Forward With Campaign to Return Mars Samples to Earth'

In [25]:
# Grab the latest teaser from NASA Mars News
nasa_teaser = soup.find_all("div", class_="article_teaser_body")[0].get_text()
nasa_teaser

'During this next phase, the program will mature critical technologies and make critical design decisions as well as assess industry partnerships.'

In [16]:
# Close the chromebrowser window
browser.quit()

In [6]:
# ADAM: this code is "test code" from Dec 11th tutoring. Probably ok to delete this before submitting final homework.
# sidebar = soup.find('div', class_='content_title')
# categories = sidebar.find_all('li')

### JPL Mars Space Images
#### Use splinter to find the current Featured Mars Image 

In [31]:
# This step opens a new chrome window that will be driven by code in the next few cells
# It also adds another Chrome icon on the taskbar because it's using chromedriver instead of chrome
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [32]:
# This step navigates the chromebrowser window to JPL images site & runs BeautifulSoup to parse the site's HTML
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)
sleep(3) # This is a manual delay that prevents a "Race Condition" with JavaScript and this script competing for resources
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [34]:
# This step clicks on the FULL IMAGE link (in the chromebrowser window)
browser.click_link_by_partial_text("FULL IMAGE")
# browser.links.find_by_partial_text("FULL IMAGE")

In [36]:
# This step clicks on the MORE INFO link (in the chromebrowser window)
sleep(3) # This is a manual delay that prevents a "Race Condition" with JavaScript and this script competing for resources
browser.click_link_by_partial_text("more info")
# browser.links.find_by_partial_text("more info")

In [37]:
# This step clicks on the JPEG image (in the chromebrowser window)
browser.click_link_by_partial_text(".jpg")

In [38]:
# This step saves as a variable the URL of the JPEG image file 
featured_image_url = browser.url
featured_image_url

'https://photojournal.jpl.nasa.gov/jpeg/PIA17171.jpg'

In [39]:
# Close the chromebrowser window
browser.quit()

### Mars Facts
#### Use Pandas to scrape the Mars Data table containing facts about the planet including Diameter, Mass, etc.

In [40]:
# Define URL of Mars Facts website
facts_url = "https://space-facts.com/mars/"
#  Read in data table via Pandas. 
# Specify zero'th position to get just Mars data, since table contains both Mars & Earth data 
facts_df = pd.read_html(facts_url)[0] 

In [41]:
# Create a "pretty" Mars Facts dataframe with column headers and text clean-up
facts_df.columns = ["Metric", "Value (Planet Mars)"]
facts_df["Metric"] = facts_df["Metric"].str.replace(":","") # Remove the colon character in DESCRIPTION column
facts_df

,Metric,Value (Planet Mars)
0,Equatorial Diameter,"6,792 km"
1,Polar Diameter,"6,752 km"
2,Mass,6.39 × 10^23 kg (0.11 Earths)
3,Moons,2 (Phobos & Deimos)
4,Orbit Distance,"227,943,824 km (1.38 AU)"
5,Orbit Period,687 days (1.9 years)
6,Surface Temperature,-87 to -5 °C
7,First Record,2nd millennium BC
8,Recorded By,Egyptian astronomers


In [42]:
# Use Pandas to convert the data to a HTML table string
facts_df.set_index("Metric", inplace=True)
facts_html = facts_df.to_html()
# print(facts_html)

### Mars Hemispheres
#### Scrape from USGS Astrogeology site hi-res images for each of Mars' hemispheres, including URL for each image.

In [61]:
# This step opens a new chrome window that will be driven by code in the next few cells
# It also adds another Chrome icon on the taskbar because it's using chromedriver instead of chrome
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [62]:
# This step navigates the chromebrowser window to USGS Astrogeology & runs BeautifulSoup to parse the site's HTML
mars_hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(mars_hemisphere_url)
sleep(3) # This is a manual delay that prevents a "Race Condition" with JavaScript and this script competing for resources
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [63]:
# Create list of Mars Hemisphere names, empty list for housing hemisphere image URLs, 
# and empty dictionary for housing hemisphere names & image URLS
mars_hemispheres = ["Cerberus","Schiaparelli","Syrtis","Valles"]
hemisphere_pic_urls = []
hemi_dict = {}

In [64]:
# Loop through list of hemispheres, and populate dictionary with name & image URL by scraping USGS site
for hemi in mars_hemispheres:
    browser.click_link_by_partial_text(hemi)
    hemi_html = browser.html
    soup = BeautifulSoup(hemi_html, 'html.parser')
    hemi_dict["title"] = soup.find("h2").get_text().replace("Enhanced","").strip()
    hemi_dict["img_url"] = soup.find_all("div", class_="downloads")[0].find_all("a")[0]["href"]
    hemisphere_pic_urls.append(hemi_dict)
    browser.back() # go back to original page with all the hemispheres

In [65]:
# Print dictionary containing Mars hemisphere names & images
hemisphere_pic_urls

[{'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [66]:
# Close the chromebrowser window
browser.quit()